In [245]:
def dataload():
    train_file = "./gowalla/train.txt"
    test_file = "./gowalla/test.txt"
    m_item = 0
    n_user = 0
    trainUniqueUsers, trainItem, trainUser = [], [], []
    testUniqueUsers, testItem, testUser = [], [], []
    with open(train_file) as f:
        for l in f.readlines():
            if len(l) > 0:
                l = l.strip('\n').split(' ')
                items = [int(i) for i in l[1:]]
                uid = int(l[0])
                trainUniqueUsers.append(uid)
                trainUser.extend([uid] * len(items))
                trainItem.extend(items)
                m_item = max(m_item, max(items))
                n_user = max(n_user, uid)
    trainUniqueUsers = np.array(trainUniqueUsers)
    trainUser = np.array(trainUser)
    trainItem = np.array(trainItem)
    return trainUser,trainItem,trainUniqueUsers
    
    

In [246]:
trainUser,trainItem,trainUniqueUsers = dataload()

In [247]:
def getgraph(trainUser,trainItem):
    g = dgl.heterograph({('user', 'edg' , 'item'): (trainUser, trainItem),
                         ('item', 'edg', 'user'): (trainItem,trainUser)})
    embedding_user = torch.nn.Embedding(g.num_nodes('user'),64)
    embedding_item = torch.nn.Embedding(g.num_nodes('item'),64)
    torch.nn.init.normal_(embedding_user.weight, std=0.1)
    torch.nn.init.normal_(embedding_item.weight, std=0.1)
    g.nodes['user'].data['emb'] =  embedding_user.weight
    g.nodes['item'].data['emb'] =  embedding_item.weight
    return g

In [251]:

class Lightgcn(nn.Module):
    def __init__(self,
                 g,
                 n_layers):
        self.g = g
        self.layers = n_layers
        self.__init_weight()
        
    def __init_weight(self):
        self.gre_user = g.out_degrees(etype=('user', 'edg1' , 'item'))
        self.gre_item = g.out_degrees(etype=('item', 'edg2' , 'user'))
        self.gre_user = gre_user ** (-0.5)
        self.gre_user = gre_user.unsqueeze(1).repeat(1,2)
        self.gre_item = gre_item  ** (-0.5)
        self.gre_item = gre_item.unsqueeze(1).repeat(1,2)

    def computer(self):
        User = []
        Item = []
        U.append(g.nodes['user'].data['emb'])
        I.append(g.nodes['item'].data['emb'])
        for i in range(self.layers):
            g.nodes['user'].data['emb'] = torch.mul(g.nodes['user'].data['emb'], self.gre_user)
            g.nodes['item'].data['emb'] = torch.mul(g.nodes['item'].data['emb'], self.gre_item)
            g.multi_update_all(
                {'edg1':(fn.copy_src('emb', 'm'), fn.sum('m', 'emb')),
                'edg2':(fn.copy_src('emb', 'm'), fn.sum('m', 'emb'))},
                "sum"
            )
            g.nodes['user'].data['emb'] = torch.mul(g.nodes['user'].data['emb'], gre_user)
            g.nodes['item'].data['emb'] = torch.mul(g.nodes['item'].data['emb'], gre_item)
            U.append(g.nodes['user'].data['emb'])
            I.append(g.nodes['item'].data['emb'])
        out_user = torch.stack(U,dim=1)
        out_user = torch.mean(out_user,dim=1)
        out_item = torch.stack(I,dim=1)
        out_item = torch.mean(out_item,dim=1)
        return out_user, out_item
    
    def bpr_loss(self)
        out_user, out_item = self.computer()
        score = torch.mm(out_user, out_item.t())
        
        
        
            
            
        
        

In [257]:


def UniformSample_original(trainUser,trainItem, g):
    n_user = g.num_nodes('user')
    m_item = g.num_nodes('item')
    UserItemNet = csr_matrix((np.ones(len(trainUser)), (trainUser, trainItem)),
                                      shape=(n_user, m_item))
    total_start = time()
    user_num = len(trainUser)
    users = np.random.randint(0, n_user, user_num)
    S = []
    sample_time1 = 0.
    sample_time2 = 0.
    allPos = []
    for user in range(n_user):
        allPos.append(UserItemNet[user].nonzero()[1])
    for i, user in enumerate(users):
        start = time()
        posForUser = allPos[user]
        if len(posForUser) == 0:
            continue
        sample_time2 += time() - start
        posindex = np.random.randint(0, len(posForUser))
        positem = posForUser[posindex]
        while True:
            negitem = np.random.randint(0, m_item)
            if negitem in posForUser:
                continue
            else:
                break
        S.append([user, positem, negitem])
        end = time()
        sample_time1 += end - start
    total = time() - total_start
    return np.array(S), [total, sample_time1, sample_time2]
    
    

In [258]:
trainUser,trainItem,trainUniqueUsers = dataload()
g = getgraph(trainUser,trainItem)
s, t = UniformSample_original(trainUser,trainItem, g)

Graph(num_nodes={'item': 40981, 'user': 29858},
      num_edges={('item', 'edg', 'user'): 810128, ('user', 'edg', 'item'): 810128},
      metagraph=[('item', 'user', 'edg'), ('user', 'item', 'edg')])